In [1]:
import json
import os
import sys
import math
import datetime
import pathlib

dir_path = pathlib.Path(os.path.abspath('')).resolve()
print(dir_path)
HOME_DIR = str(dir_path).split('/OpenFormat')[0]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

PROJ_SRC_DIR = f'{HOME_DIR}/OpenFormat'
sys.path.insert(1, f'{PROJ_SRC_DIR}')
from python.scripts.utils import *
num_rows = 1000 * 1000
num_cols = 20
workload_list = ['int', 'float', 'string']
scan_exec_pq = f'{HOME_DIR}/arrow-private/cpp/out/build/openformat-release/release/parquet-scan-columnbatch'

/mnt/OpenFormat/python/encoding


In [2]:
%%time
# generate csv files
os.chdir(f'{PROJ_SRC_DIR}/benchmark/generator_v2')
!pwd
for wl in workload_list:
    base_name = f'{wl}_r{num_rows}_c{num_cols}'
    os.system(f'python3 gen_workloads.py \
        {wl} {num_rows} {num_cols} {base_name}')
    os.system(f'cp -r {base_name} {dir_path} && \
        rm -r {base_name}')
os.chdir(dir_path)
!pwd


/mnt/OpenFormat/benchmark/generator_v2
🤔 INFO: too many collisions in int_unique_list, 20597 values just use (max_value + 1)!
🤔 INFO: too many collisions in int_unique_list, 7417 values just use (max_value + 1)!
Finish core dataset! 🎉🎉🎉🎉🎉🎉
Generating Done!!
Finish core dataset! 🎉🎉🎉🎉🎉🎉
Generating Done!!
Finish core dataset! 🎉🎉🎉🎉🎉🎉
Generating Done!!
/mnt/OpenFormat/python/encoding
CPU times: user 15.3 ms, sys: 11.4 ms, total: 26.8 ms
Wall time: 6min 31s


In [4]:
pq_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/pq_encoding.json')
orc_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/orc_encoding.json')
pq_name_suffix = '_' + "_".join([str(i) for i in list(pq_config[0].values())]) + '.parquet'
orc_name_suffix = '_' + "_".join([str(i) for i in list(orc_config[0].values())]) + '.orc'
print(pq_config)

loading config from /mnt/OpenFormat/python/experiments/pq_encoding.json ...
loading config from /mnt/OpenFormat/python/experiments/orc_encoding.json ...
[{'compression': 'NONE', 'use_dictionary': True, 'dictionary_pagesize_limit': 2147483647, 'version': '2.6', 'data_page_version': '2.0'}]


In [4]:
os.chdir(f'{PROJ_SRC_DIR}/python')
# workload_lists = ['core', 'geo', 'classic', 'log', 'ml']
for wl in workload_list:
    base_name = f'{wl}_r{num_rows}_c{num_cols}'
    os.system(f'cp encoding/{base_name}/gen_data/{base_name}.csv data_gen')
    os.system(f'python3 scripts/parquet_exp.py {base_name} pq_encoding -r=false -o=false')
    os.system(f'python3 scripts/orc_exp.py {base_name} orc_encoding -r=false -o=false')
    # orc_schema = open(f'encoding/{base_name}/gen_data/{base_name}_orc_schema.txt', 'r').read()
    # os.system(f'{HOME_DIR}/orc/build/tools/src/csv-import \"{orc_schema}\" \
    #         encoding/{base_name}/gen_data/{base_name}.csv {base_name}.orc')
    os.system(f'mv {base_name}{pq_name_suffix} encoding/{base_name}.parquet')
    os.system(f'mv {base_name}{orc_name_suffix} encoding/{base_name}.orc')
    # os.system(f'mv {base_name}.orc encoding/{base_name}.orc')
os.chdir(f'{PROJ_SRC_DIR}/python/encoding')

loading config from ./experiments/pq_encoding.json ...
----Running experiment: 1 out of 1----
NONE_True_2147483647_2.6_2.0
write parquet time (s): 0.5347259044647217
loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 0.7598474025726318
loading config from ./experiments/pq_encoding.json ...
----Running experiment: 1 out of 1----
NONE_True_2147483647_2.6_2.0
write parquet time (s): 0.7709953784942627
loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 0.6040549278259277
loading config from ./experiments/pq_encoding.json ...
----Running experiment: 1 out of 1----
NONE_True_2147483647_2.6_2.0
write parquet time (s): 1.8646364212036133
loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 11.64716649055481


# RLE breakdown

In [2]:
run_list = [2, 3, 4, 5,6,7,8, 9,10]

In [5]:
%time
os.chdir(f'{PROJ_SRC_DIR}/python/data_gen')
for run in run_list:
    os.system(f'python3 gen_simple_short_repeat.py {run}')
os.chdir(f'{PROJ_SRC_DIR}/python')
for run in run_list:
    base_name = f'idx_run{run}_max9'
    os.system(f'python3 scripts/parquet_exp.py {base_name} pq_encoding -r=false -o=false')
    os.system(f'python3 scripts/orc_exp.py {base_name} orc_encoding -r=false -o=false')
    # orc_schema = open(f'encoding/{base_name}/gen_data/{base_name}_orc_schema.txt', 'r').read()
    # os.system(f'{HOME_DIR}/orc/build/tools/src/csv-import \"{orc_schema}\" \
    #         encoding/{base_name}/gen_data/{base_name}.csv {base_name}.orc')
    os.system(f'mv {base_name}{pq_name_suffix} encoding/{base_name}.parquet')
    os.system(f'mv {base_name}{orc_name_suffix} encoding/{base_name}.orc')
    # os.system(f'mv {base_name}.orc encoding/{base_name}.orc')
os.chdir(f'{PROJ_SRC_DIR}/python/encoding')


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.44 µs
loading config from ./experiments/pq_encoding.json ...
----Running experiment: 1 out of 1----
NONE_True_2147483647_2.6_2.0
write parquet time (s): 0.9392552375793457
loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 3.182405710220337
loading config from ./experiments/pq_encoding.json ...
----Running experiment: 1 out of 1----
NONE_True_2147483647_2.6_2.0
write parquet time (s): 0.927161693572998
loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 2.820885419845581
loading config from ./experiments/pq_encoding.json ...
----Running experiment: 1 out of 1----
NONE_True_2147483647_2.6_2.0
write parquet time (s): 0.922966718673706
loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (

In [6]:
# running experiments
os.system('rm outputs/stats.json')
for i in range(10):
    for run in run_list:
        base_name = f'idx_run{run}_max9'
        os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
        time.sleep(1)
        pq_read = float(os.popen(f'''{scan_exec_pq} \
                --batch_size=1024 ./{base_name}.parquet''').read().split('\n')[0].split(' ')[-2])
        pq_read_preload = float(os.popen(f'''{scan_exec_pq} \
                --batch_size=1024 ./{base_name}.parquet''').read().split('\n')[0].split(' ')[-2])
        os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
        time.sleep(1)
        orc_read = float(os.popen(f'''{HOME_DIR}/orc/build/tools/src/orc-scan \
                -b 1024 ./{base_name}.orc''').read().split('\n')[0].split(' ')[-1])
        orc_read_preload = float(os.popen(f'''{HOME_DIR}/orc/build/tools/src/orc-scan \
                -b 1024 ./{base_name}.orc''').read().split('\n')[0].split(' ')[-1])
        orc_size = os.path.getsize(f'{base_name}.orc')
        pq_size = os.path.getsize(f'{base_name}.parquet')
        output_stats = {}
        output_stats['run'] = run
        output_stats['i'] = i
        output_stats['file'] = 'parquet'
        output_stats['size'] = pq_size
        output_stats['read_time'] = pq_read
        output_stats['read_time_preload'] = pq_read_preload
        parse_output(output_stats)
        output_stats['file'] = 'orc'
        output_stats['size'] = orc_size
        output_stats['read_time'] = orc_read
        output_stats['read_time_preload'] = orc_read_preload
        parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format('decode_rle', timestamp))

0

# Decode time on varying compressions

In [10]:
# pq_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/pq_default.json')
# orc_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/orc_default.json')
pq_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/pq_encoding.json')
orc_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/orc_encoding.json')
pq_param = pq_config[0]
orc_param = orc_config[0]
import pyarrow.orc as po
from pyarrow import csv
import pyarrow.parquet as pq
os.chdir(f'{PROJ_SRC_DIR}/python')
workload_list = ['int', 'float', 'string']
compression_list = ['SNAPPY', 'ZSTD', 'LZ4', 'NONE']
for compression in compression_list:
    orc_param['compression'] = compression if compression != 'NONE' else "UNCOMPRESSED"
    pq_param['compression'] = compression
    for wl in workload_list:
        base_name = f'{wl}_r{num_rows}_c{num_cols}'
        table, _ = read_csv(f'encoding/{base_name}/gen_data/{base_name}.csv')
        po.write_table(
            table, f'encoding/{base_name}.{compression.lower()}.orc', **orc_param)
        pq.write_table(table, f'encoding/{base_name}.{compression.lower()}.parquet', 
                **pq_param)
        # os.system(f'mv {base_name}.orc encoding/{base_name}.orc')
os.chdir(f'{PROJ_SRC_DIR}/python/encoding')

loading config from /mnt/OpenFormat/python/experiments/pq_encoding.json ...
loading config from /mnt/OpenFormat/python/experiments/orc_encoding.json ...


In [14]:
# running experiments
os.system('rm outputs/stats.json')
for i in range(10):
    for c in compression_list:
        for wl in workload_list:
                base_name = f'{wl}_r{num_rows}_c{num_cols}'
                os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
                time.sleep(1)
                pq_read = float(os.popen(f'''{scan_exec_pq} \
                        --batch_size=1024 ./{base_name}.{c.lower()}.parquet''').read().split('\n')[0].split(' ')[-2])
                pq_read_preload = float(os.popen(f'''{scan_exec_pq} \
                        --batch_size=1024 ./{base_name}.{c.lower()}.parquet''').read().split('\n')[0].split(' ')[-2])
                os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
                time.sleep(1)
                orc_read = float(os.popen(f'''{HOME_DIR}/orc/build/tools/src/orc-scan \
                        -b 1024 ./{base_name}.{c.lower()}.orc''').read().split('\n')[0].split(' ')[-1])
                orc_read_preload = float(os.popen(f'''{HOME_DIR}/orc/build/tools/src/orc-scan \
                        -b 1024 ./{base_name}.{c.lower()}.orc''').read().split('\n')[0].split(' ')[-1])
                orc_size = os.path.getsize(f'{base_name}.{c.lower()}.orc')
                pq_size = os.path.getsize(f'{base_name}.{c.lower()}.parquet')
                output_stats = {}
                output_stats['workload'] = wl
                output_stats['i'] = i
                output_stats['file'] = 'parquet'
                output_stats['size'] = pq_size
                output_stats['read_time'] = pq_read
                output_stats['read_time_preload'] = pq_read_preload
                output_stats['compression'] = c.lower()
                parse_output(output_stats)
                output_stats['file'] = 'orc'
                output_stats['size'] = orc_size
                output_stats['read_time'] = orc_read
                output_stats['read_time_preload'] = orc_read_preload
                output_stats['compression'] = c.lower()
                parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format('decode_speed_compression', timestamp))

0